# Recommendation Process

In [1]:
import pandas as pd
import numpy as np
import cv2
import pytesseract

In [2]:
df = pd.read_csv('classification_data.csv')

In [3]:
df

,count_session,count_hit,channelGrouping,totals_newVisits,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_region,geoNetwork_latitude,...,latest_keyword,earliest_isTrueDirect,latest_isTrueDirect,num_interactions,bounces,time_on_site,transactionRevenue,products_array,has_converted,target_date
0,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food////food////food,0,2021-01-01
1,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food////food////food,0,2021-01-01
2,28,1816,Paid Search,0,GoogleAnalytics,Android,True,mobile,Ajman,25.4052,...,(automatic matching),True,True,3843,0,77417,1477195214,"(not set)////baby_products--milk,_food_&_juice...",1,2021-01-01
3,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food////food,0,2021-01-01
4,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food////food,0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby_products--milk,_food_&_juices--baby_&_tod...",1,2021-01-01
99996,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby_products--milk,_food_&_juices--baby_&_tod...",1,2021-01-01
99997,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby_products--milk,_food_&_juices--baby_&_tod...",1,2021-01-01
99998,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby_products--milk,_food_&_juices--baby_&_tod...",1,2021-01-01


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 46 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   count_session           100000 non-null  int64  
 1   count_hit               100000 non-null  int64  
 2   channelGrouping         100000 non-null  object 
 3   totals_newVisits        100000 non-null  int64  
 4   device_browser          100000 non-null  object 
 5   device_operatingSystem  100000 non-null  object 
 6   device_isMobile         100000 non-null  bool   
 7   device_deviceCategory   100000 non-null  object 
 8   geoNetwork_region       100000 non-null  object 
 9   geoNetwork_latitude     100000 non-null  float64
 10  geoNetwork_longitude    100000 non-null  float64
 11  historic_session        100000 non-null  int64  
 12  historic_session_page   100000 non-null  int64  
 13  youtube                 100000 non-null  int64  
 14  avg_session_time     

In [5]:
df.dtypes

count_session               int64
count_hit                   int64
channelGrouping            object
totals_newVisits            int64
device_browser             object
device_operatingSystem     object
device_isMobile              bool
device_deviceCategory      object
geoNetwork_region          object
geoNetwork_latitude       float64
geoNetwork_longitude      float64
historic_session            int64
historic_session_page       int64
youtube                     int64
avg_session_time          float64
avg_session_time_page     float64
single_page_rate          float64
sessionQualityDim           int64
last_visitId                int64
latest_visit_id             int64
visitId_threshold           int64
earliest_visit_id           int64
earliest_visit_number       int64
latest_visit_number         int64
time_earliest_visit       float64
time_latest_visit         float64
avg_visit_time              int64
days_since_last_visit       int64
days_since_first_visit      int64
visits_per_day

In [6]:
df.columns

Index(['count_session', 'count_hit', 'channelGrouping', 'totals_newVisits',
       'device_browser', 'device_operatingSystem', 'device_isMobile',
       'device_deviceCategory', 'geoNetwork_region', 'geoNetwork_latitude',
       'geoNetwork_longitude', 'historic_session', 'historic_session_page',
       'youtube', 'avg_session_time', 'avg_session_time_page',
       'single_page_rate', 'sessionQualityDim', 'last_visitId',
       'latest_visit_id', 'visitId_threshold', 'earliest_visit_id',
       'earliest_visit_number', 'latest_visit_number', 'time_earliest_visit',
       'time_latest_visit', 'avg_visit_time', 'days_since_last_visit',
       'days_since_first_visit', 'visits_per_day', 'bounce_rate',
       'earliest_source', 'latest_source', 'earliest_medium', 'latest_medium',
       'earliest_keyword', 'latest_keyword', 'earliest_isTrueDirect',
       'latest_isTrueDirect', 'num_interactions', 'bounces', 'time_on_site',
       'transactionRevenue', 'products_array', 'has_converted', 't

In [7]:
columns_to_drop=['youtube', 'days_since_last_visit','earliest_visit_id','latest_visit_id','last_visitId','time_earliest_visit','time_latest_visit','latest_visit_number',]

In [8]:
df = df.drop(columns=columns_to_drop)

In [9]:
df.columns

Index(['count_session', 'count_hit', 'channelGrouping', 'totals_newVisits',
       'device_browser', 'device_operatingSystem', 'device_isMobile',
       'device_deviceCategory', 'geoNetwork_region', 'geoNetwork_latitude',
       'geoNetwork_longitude', 'historic_session', 'historic_session_page',
       'avg_session_time', 'avg_session_time_page', 'single_page_rate',
       'sessionQualityDim', 'visitId_threshold', 'earliest_visit_number',
       'avg_visit_time', 'days_since_first_visit', 'visits_per_day',
       'bounce_rate', 'earliest_source', 'latest_source', 'earliest_medium',
       'latest_medium', 'earliest_keyword', 'latest_keyword',
       'earliest_isTrueDirect', 'latest_isTrueDirect', 'num_interactions',
       'bounces', 'time_on_site', 'transactionRevenue', 'products_array',
       'has_converted', 'target_date'],
      dtype='object')

In [10]:
new_df = df.copy()

In [11]:
new_df

,count_session,count_hit,channelGrouping,totals_newVisits,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_region,geoNetwork_latitude,...,latest_keyword,earliest_isTrueDirect,latest_isTrueDirect,num_interactions,bounces,time_on_site,transactionRevenue,products_array,has_converted,target_date
0,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food////food////food,0,2021-01-01
1,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food////food////food,0,2021-01-01
2,28,1816,Paid Search,0,GoogleAnalytics,Android,True,mobile,Ajman,25.4052,...,(automatic matching),True,True,3843,0,77417,1477195214,"(not set)////baby_products--milk,_food_&_juice...",1,2021-01-01
3,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food////food,0,2021-01-01
4,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food////food,0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby_products--milk,_food_&_juices--baby_&_tod...",1,2021-01-01
99996,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby_products--milk,_food_&_juices--baby_&_tod...",1,2021-01-01
99997,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby_products--milk,_food_&_juices--baby_&_tod...",1,2021-01-01
99998,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby_products--milk,_food_&_juices--baby_&_tod...",1,2021-01-01


In [191]:
pd.set_option('display.max_colwidth',3000)

In [187]:
def clean_products_array_column(df):
    df['products_array'] = df['products_array'].str.replace('////', ',').str.replace('--', ',').str.replace('_', ' ')

# Assuming new_df is your DataFrame
clean_products_array_column(new_df)

new_df['products_array']

0                                           food,food,food
1                                           food,food,food
2        (not set),baby products,milk, food & juices,ba...
3                                                food,food
4                                                food,food
                               ...                        
99995    baby products,milk, food & juices,baby & toddl...
99996    baby products,milk, food & juices,baby & toddl...
99997    baby products,milk, food & juices,baby & toddl...
99998    baby products,milk, food & juices,baby & toddl...
99999    baby products,milk, food & juices,baby & toddl...
Name: products_array, Length: 100000, dtype: object

In [192]:
new_df['products_array']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        food,food,food
1                                                                                                                                                                                                                                                                               

In [200]:
column_to_replace = 'products_array'

# Replace ',(not set)' with an empty string
new_df[column_to_replace] = new_df[column_to_replace].str.replace(',(not set)', '')


In [189]:
def clean_products_array_column(df):
    # Replace '(not set),' with an empty string
    df['products_array'] = df['products_array'].str.replace('(not set),', '')
    
    # Replace '(not set)' with None
    df['products_array'] = df['products_array'].replace({'(not set)': None})

# Assuming new_df is your DataFrame
clean_products_array_column(new_df)

new_df['products_array']

0                                           food,food,food
1                                           food,food,food
2        baby products,milk, food & juices,baby & toddl...
3                                                food,food
4                                                food,food
                               ...                        
99995    baby products,milk, food & juices,baby & toddl...
99996    baby products,milk, food & juices,baby & toddl...
99997    baby products,milk, food & juices,baby & toddl...
99998    baby products,milk, food & juices,baby & toddl...
99999    baby products,milk, food & juices,baby & toddl...
Name: products_array, Length: 100000, dtype: object


In [201]:
new_df.head(30)

,count_session,count_hit,channelGrouping,totals_newVisits,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_region,geoNetwork_latitude,...,latest_keyword,earliest_isTrueDirect,latest_isTrueDirect,num_interactions,bounces,time_on_site,transactionRevenue,products_array,has_converted,target_date
0,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,"food,food,food",0,2021-01-01
1,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,"food,food,food",0,2021-01-01
2,28,1816,Paid Search,0,GoogleAnalytics,Android,True,mobile,Ajman,25.4052,...,(automatic matching),True,True,3843,0,77417,1477195214,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),baby products,milk, food & juices,baby & toddler formula,baby formula (1-6 years),food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),food,food,food,food,food,food,baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),food,food,food,baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food,food",1,2021-01-01
3,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,"food,food",0,2021-01-01
4,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,"food,food",0,2021-01-01
5,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,"food,food",0,2021-01-01
6,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,"food,food",0,2021-01-01
7,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,"food,food",0,2021-01-01
8,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,"food,food",0,2021-01-01
9,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,"food,food",0,2021-01-01


In [202]:
column_to_clean = 'products_array'

# Define a function to remove duplicates within each row
def remove_duplicates(row):
    return ','.join(pd.unique(row.split(',')))

# Apply the function to the specified column
new_df[column_to_clean] = new_df[column_to_clean].apply(remove_duplicates)

new_df

       count_session  count_hit channelGrouping  totals_newVisits  \
0                  9       1194         (Other)                 0   
1                  9       1194         (Other)                 0   
2                 28       1816     Paid Search                 0   
3                  2        299     Paid Search                 0   
4                  2        299     Paid Search                 0   
...              ...        ...             ...               ...   
99995             58      12340     Paid Search                 0   
99996             58      12340     Paid Search                 0   
99997             58      12340     Paid Search                 0   
99998             58      12340     Paid Search                 0   
99999             58      12340     Paid Search                 0   

        device_browser device_operatingSystem  device_isMobile  \
0      GoogleAnalytics                Android             True   
1      GoogleAnalytics                A

In [205]:
new_df.dropna(subset=['products_array'], inplace=True)

In [207]:
new_df['products_array'].replace('', np.nan, inplace=True)

In [210]:
new_df= new_df.dropna(subset=['products_array'])

In [206]:
new_df

,count_session,count_hit,channelGrouping,totals_newVisits,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_region,geoNetwork_latitude,...,latest_keyword,earliest_isTrueDirect,latest_isTrueDirect,num_interactions,bounces,time_on_site,transactionRevenue,products_array,has_converted,target_date
0,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food,0,2021-01-01
1,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food,0,2021-01-01
2,28,1816,Paid Search,0,GoogleAnalytics,Android,True,mobile,Ajman,25.4052,...,(automatic matching),True,True,3843,0,77417,1477195214,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),baby formula (1-6 years),food",1,2021-01-01
3,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food,0,2021-01-01
4,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food,0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99996,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99997,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99998,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01


In [212]:
new_df.columns

Index(['count_session', 'count_hit', 'channelGrouping', 'totals_newVisits',
       'device_browser', 'device_operatingSystem', 'device_isMobile',
       'device_deviceCategory', 'geoNetwork_region', 'geoNetwork_latitude',
       'geoNetwork_longitude', 'historic_session', 'historic_session_page',
       'avg_session_time', 'avg_session_time_page', 'single_page_rate',
       'sessionQualityDim', 'visitId_threshold', 'earliest_visit_number',
       'avg_visit_time', 'days_since_first_visit', 'visits_per_day',
       'bounce_rate', 'earliest_source', 'latest_source', 'earliest_medium',
       'latest_medium', 'earliest_keyword', 'latest_keyword',
       'earliest_isTrueDirect', 'latest_isTrueDirect', 'num_interactions',
       'bounces', 'time_on_site', 'transactionRevenue', 'products_array',
       'has_converted', 'target_date'],
      dtype='object')

In [213]:
df_1 = new_df[["visitId_threshold","products_array","target_date"]]

In [211]:
new_df

,count_session,count_hit,channelGrouping,totals_newVisits,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_region,geoNetwork_latitude,...,latest_keyword,earliest_isTrueDirect,latest_isTrueDirect,num_interactions,bounces,time_on_site,transactionRevenue,products_array,has_converted,target_date
0,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food,0,2021-01-01
1,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food,0,2021-01-01
2,28,1816,Paid Search,0,GoogleAnalytics,Android,True,mobile,Ajman,25.4052,...,(automatic matching),True,True,3843,0,77417,1477195214,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),baby formula (1-6 years),food",1,2021-01-01
3,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food,0,2021-01-01
4,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food,0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99996,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99997,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99998,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01


In [256]:
df_1

,visitId_threshold,products_array,target_date
0,1617217803,food,2021-01-01
1,1617217803,food,2021-01-01
2,1617275532,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),baby formula (1-6 years),food",2021-01-01
3,1615301676,food,2021-01-01
4,1615301676,food,2021-01-01
...,...,...,...
99995,1617198770,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",2021-01-01
99996,1617198770,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",2021-01-01
99997,1617198770,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",2021-01-01
99998,1617198770,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",2021-01-01


In [257]:
df_1['products_array'] = df_1['products_array'].str.split(r'\s*,\s*')

/var/folders/5_/gr87l7990pq6c1zhr9t2t40r0000gn/T/ipykernel_6182/3091701018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['products_array'] = df_1['products_array'].str.split(r'\s*,\s*')


In [273]:
data = df_1.drop_duplicates(subset=['visitId_threshold'])

In [331]:
data = data.reset_index(drop=True)

In [332]:
data

,VisitId,Products,Date
0,1617217803,[food],2021-01-01
1,1617275532,"[baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years), baby formula (1-6 years), food]",2021-01-01
2,1615301676,[food],2021-01-01
3,1613977212,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]",2021-01-01
4,1617089505,"[food, baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]",2021-01-01
...,...,...,...
8053,1616528197,[food],2021-01-01
8054,1617194911,[food],2021-01-01
8055,1617193032,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, milk, food & juices, baby & toddler formula, baby formula (0-6 months), pull-ups & swimming pants, baby pants]",2021-01-01
8056,1616641350,[food],2021-01-01


## Apriori

In [334]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Initialize TransactionEncoder
te = TransactionEncoder()

# Transform the data into a one-hot encoded DataFrame
one_hot_encoded = te.fit(data['Products']).transform(data['Products'])

# Convert to a pandas DataFrame
df_encoded = pd.DataFrame(one_hot_encoded, columns=te.columns_)

# Run Apriori algorithm
frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# Display the resulting rules
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(baby & toddler formula),(baby products),0.268553,0.451973,0.268553,1.000000,2.212521,0.147174,inf,0.749237
1,(food & juices),(baby & toddler formula),0.269298,0.268553,0.268553,0.997235,3.713364,0.196232,264.540002,1.000000
2,(baby & toddler formula),(food & juices),0.268553,0.269298,0.268553,1.000000,3.713364,0.196232,inf,0.998982
3,(baby & toddler formula),(milk),0.268553,0.271283,0.268553,1.000000,3.686185,0.195699,inf,0.996267
4,(milk),(baby & toddler formula),0.271283,0.268553,0.268553,0.989936,3.686185,0.195699,72.679235,1.000000
5,(changing & bathing),(baby products),0.273517,0.451973,0.273517,1.000000,2.212521,0.149895,inf,0.754356
6,(food & juices),(baby products),0.269298,0.451973,0.269298,1.000000,2.212521,0.147582,inf,0.750000
7,(milk),(baby products),0.271283,0.451973,0.271283,1.000000,2.212521,0.148670,inf,0.752044
8,(food & juices),(milk),0.269298,0.271283,0.269298,1.000000,3.686185,0.196242,inf,0.997283
9,(milk),(food & juices),0.271283,0.269298,0.269298,0.992681,3.686185,0.196242,99.832216,1.000000


In [335]:
df_encoded

,Baby Diapers,Baby Pants,Baby Products,Baby and Personal Care,Bulk Savings,Changing & Bathing,Changing &amp; Bathing,Changing &amp;amp; Bathing,Diapers,Pull-ups & Swimming Pants,...,powdered milk,probiotic yoghurt,pull-ups & swimming pants,pull-ups &amp; swimming pants,tins,wipes & diaper cream,wipes &amp; diaper cream,yoghurt,حفاضات اطفال,سراويل الأطفال
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8053,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8054,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8055,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
8056,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## fpgrowth

In [336]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules


# Initialize TransactionEncoder
te = TransactionEncoder()

# Transform the data into a one-hot encoded DataFrame
one_hot_encoded = te.fit(data['Products']).transform(data['Products'])

# Convert to a pandas DataFrame
df_encoded1 = pd.DataFrame(one_hot_encoded, columns=te.columns_)

# Run FP-growth algorithm
frequent_itemsets = fpgrowth(df_encoded1, min_support=0.2, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# Display the resulting rules
rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(milk),(baby products),0.271283,0.451973,0.271283,1.000000,2.212521,0.148670,inf,0.752044
1,(food & juices),(milk),0.269298,0.271283,0.269298,1.000000,3.686185,0.196242,inf,0.997283
2,(milk),(food & juices),0.271283,0.269298,0.269298,0.992681,3.686185,0.196242,99.832216,1.000000
3,(food & juices),(baby products),0.269298,0.451973,0.269298,1.000000,2.212521,0.147582,inf,0.750000
4,"(food & juices, milk)",(baby products),0.269298,0.451973,0.269298,1.000000,2.212521,0.147582,inf,0.750000
5,"(food & juices, baby products)",(milk),0.269298,0.271283,0.269298,1.000000,3.686185,0.196242,inf,0.997283
6,"(milk, baby products)",(food & juices),0.271283,0.269298,0.269298,0.992681,3.686185,0.196242,99.832216,1.000000
7,(food & juices),"(milk, baby products)",0.269298,0.271283,0.269298,1.000000,3.686185,0.196242,inf,0.997283
8,(milk),"(food & juices, baby products)",0.271283,0.269298,0.269298,0.992681,3.686185,0.196242,99.832216,1.000000
9,(food & juices),(baby & toddler formula),0.269298,0.268553,0.268553,0.997235,3.713364,0.196232,264.540002,1.000000


In [337]:
df_encoded1

,Baby Diapers,Baby Pants,Baby Products,Baby and Personal Care,Bulk Savings,Changing & Bathing,Changing &amp; Bathing,Changing &amp;amp; Bathing,Diapers,Pull-ups & Swimming Pants,...,powdered milk,probiotic yoghurt,pull-ups & swimming pants,pull-ups &amp; swimming pants,tins,wipes & diaper cream,wipes &amp; diaper cream,yoghurt,حفاضات اطفال,سراويل الأطفال
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8053,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8054,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8055,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
8056,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [338]:
data.columns

Index(['VisitId', 'Products', 'Date'], dtype='object')

In [307]:
# data.columns = ['VisitId','Products','Date']

In [323]:
# data = data.reset_index(drop=True)

In [340]:
data.head()

,VisitId,Products,Date
0,1617217803,[food],2021-01-01
1,1617275532,"[baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years), baby formula (1-6 years), food]",2021-01-01
2,1615301676,[food],2021-01-01
3,1613977212,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]",2021-01-01
4,1617089505,"[food, baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]",2021-01-01


In [319]:
newdf = new_df.drop_duplicates(subset=['visitId_threshold']).reset_index(drop=True)

In [322]:
newdf.head()

,count_session,count_hit,channelGrouping,totals_newVisits,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_region,geoNetwork_latitude,...,latest_keyword,earliest_isTrueDirect,latest_isTrueDirect,num_interactions,bounces,time_on_site,transactionRevenue,products_array,has_converted,target_date
0,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food,0,2021-01-01
1,28,1816,Paid Search,0,GoogleAnalytics,Android,True,mobile,Ajman,25.4052,...,(automatic matching),True,True,3843,0,77417,1477195214,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),baby formula (1-6 years),food",1,2021-01-01
2,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food,0,2021-01-01
3,3,507,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace,True,True,507,0,1958,28745286,"baby products,changing & bathing,diapers, wipes & diaper cream,baby diapers",1,2021-01-01
4,9,1037,Paid Search,0,GoogleAnalytics,Android,True,mobile,Dubai,25.2048,...,(not set),True,True,1674,0,138440,1054124685,"food,baby products,milk, food & juices,baby & toddler formula,baby formula (0-6 months)",1,2021-01-01


In [315]:
newdf.columns

Index(['count_session', 'count_hit', 'channelGrouping', 'totals_newVisits',
       'device_browser', 'device_operatingSystem', 'device_isMobile',
       'device_deviceCategory', 'geoNetwork_region', 'geoNetwork_latitude',
       'geoNetwork_longitude', 'historic_session', 'historic_session_page',
       'avg_session_time', 'avg_session_time_page', 'single_page_rate',
       'sessionQualityDim', 'visitId_threshold', 'earliest_visit_number',
       'avg_visit_time', 'days_since_first_visit', 'visits_per_day',
       'bounce_rate', 'earliest_source', 'latest_source', 'earliest_medium',
       'latest_medium', 'earliest_keyword', 'latest_keyword',
       'earliest_isTrueDirect', 'latest_isTrueDirect', 'num_interactions',
       'bounces', 'time_on_site', 'transactionRevenue', 'products_array',
       'has_converted', 'target_date'],
      dtype='object')

In [360]:
new_df

,count_session,count_hit,channelGrouping,totals_newVisits,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_region,geoNetwork_latitude,...,latest_keyword,earliest_isTrueDirect,latest_isTrueDirect,num_interactions,bounces,time_on_site,transactionRevenue,products_array,has_converted,target_date
0,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food,0,2021-01-01
1,9,1194,(Other),0,GoogleAnalytics,Android,True,mobile,Sharjah,25.3463,...,(not set),True,True,1900,0,5617,0,food,0,2021-01-01
2,28,1816,Paid Search,0,GoogleAnalytics,Android,True,mobile,Ajman,25.4052,...,(automatic matching),True,True,3843,0,77417,1477195214,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),baby formula (1-6 years),food",1,2021-01-01
3,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food,0,2021-01-01
4,2,299,Paid Search,0,GoogleAnalytics,Android,True,mobile,Abu Dhabi,24.4539,...,custom0==marketplace&+id==*,True,True,592,0,1576,0,food,0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99996,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99997,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01
99998,58,12340,Paid Search,0,GoogleAnalytics,iOS,True,mobile,Dubai,25.2048,...,custom0==marketplace&+id==*,True,True,34999,0,0,149151662,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",1,2021-01-01


In [363]:
new_df['single_page_rate'].value_counts()

single_page_rate
1.000000    55336
0.000000     3550
0.800000      963
0.833333      921
0.666667      868
            ...  
0.853659        1
0.980392        1
0.990196        1
0.932692        1
0.901961        1
Name: count, Length: 466, dtype: int64

In [364]:
new_df['single_page_rate'].count()

85290

In [372]:
new_df['single_page_rate'].unique()

array([0.77777778, 0.82142857, 1.        , 0.4       , 0.98701299,
       0.92857143, 0.96296296, 0.25      , 0.75      , 0.66666667,
       0.175     , 0.88888889, 0.86363636, 0.97058824, 0.98507463,
       0.91428571, 0.        , 0.93333333, 0.97368421, 0.71428571,
       0.8       , 0.93589744, 0.81818182, 0.94444444, 0.5       ,
       0.90909091, 0.83333333, 0.9       , 0.97560976, 0.89473684,
       0.96666667, 0.41666667, 0.125     , 0.95238095, 0.76923077,
       0.16666667, 0.94814815, 0.78571429, 0.94117647, 0.95833333,
       0.92307692, 0.88235294, 0.76470588, 0.96551724, 0.80645161,
       0.28571429, 0.95      , 0.85714286, 0.93181818, 0.2       ,
       0.78787879, 0.625     , 0.63333333, 0.88461538, 0.93548387,
       0.97727273, 0.73913043, 0.76296296, 0.95918367, 0.875     ,
       0.98113208, 0.36363636, 0.95454545, 0.87037037, 0.61111111,
       0.7       , 0.94736842, 0.95652174, 0.33333333, 0.94366197,
       0.91525424, 0.78947368, 0.34615385, 0.77419355, 0.72916

In [361]:
df_2 = new_df[['visitId_threshold','products_array','target_date','single_page_rate']]

In [362]:
df_2

,visitId_threshold,products_array,target_date,single_page_rate
0,1617217803,food,2021-01-01,0.777778
1,1617217803,food,2021-01-01,0.777778
2,1617275532,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 years),baby formula (1-6 years),food",2021-01-01,0.821429
3,1615301676,food,2021-01-01,1.000000
4,1615301676,food,2021-01-01,1.000000
...,...,...,...,...
99995,1617198770,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",2021-01-01,1.000000
99996,1617198770,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",2021-01-01,1.000000
99997,1617198770,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",2021-01-01,1.000000
99998,1617198770,"baby products,milk, food & juices,baby & toddler formula,baby formula (6-12 months)",2021-01-01,1.000000


In [365]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85290 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   visitId_threshold  85290 non-null  int64  
 1   products_array     85290 non-null  object 
 2   target_date        85290 non-null  object 
 3   single_page_rate   85290 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 5.3+ MB


In [366]:
df_2 = df_2.sort_values("visitId_threshold", ascending = True)

In [367]:
df_2

,visitId_threshold,products_array,target_date,single_page_rate
71948,1606899638,"baby products,milk, food & juices,baby & toddler formula,baby formula (1-6 years)",2021-01-01,1.000000
71949,1606899638,"baby products,milk, food & juices,baby & toddler formula,baby formula (1-6 years)",2021-01-01,1.000000
71950,1606899638,"baby products,milk, food & juices,baby & toddler formula,baby formula (1-6 years)",2021-01-01,1.000000
38893,1606899665,"baby products,changing & bathing,diapers, wipes & diaper cream,baby diapers",2021-01-01,0.866667
38894,1606899665,"baby products,changing & bathing,diapers, wipes & diaper cream,baby diapers",2021-01-01,0.866667
...,...,...,...,...
84302,1617307178,"baby products,changing & bathing,diapers, wipes & diaper cream,baby diapers,milk, food & juices,baby & toddler formula,baby formula (6-12 months),baby formula (0-6 months)",2021-01-01,1.000000
84303,1617307178,"baby products,changing & bathing,diapers, wipes & diaper cream,baby diapers,milk, food & juices,baby & toddler formula,baby formula (6-12 months),baby formula (0-6 months)",2021-01-01,1.000000
36752,1617307194,"baby products,changing & bathing,diapers, wipes & diaper cream,baby diapers,milk, food & juices,baby & toddler formula,baby formula (1-6 years),pull-ups & swimming pants,baby pants",2021-01-01,1.000000
36751,1617307194,"baby products,changing & bathing,diapers, wipes & diaper cream,baby diapers,milk, food & juices,baby & toddler formula,baby formula (1-6 years),pull-ups & swimming pants,baby pants",2021-01-01,1.000000


In [368]:
df_2 = df_2.groupby("visitId_threshold")['single_page_rate'].sum().reset_index()

In [370]:
df_2['single_page_rate'] = df_2['single_page_rate'].round(1)

In [371]:
df_2

,visitId_threshold,single_page_rate
0,1606899638,3.0
1,1606899665,10.4
2,1606922803,57.0
3,1606959100,4.0
4,1607000156,0.0
...,...,...
8053,1617307082,1.5
8054,1617307117,1.0
8055,1617307129,1.0
8056,1617307178,2.0


In [375]:
df_3 = data.sort_values("VisitId", ascending = True).reset_index(drop=True)

In [377]:
df_3['Rating'] = df_2['single_page_rate']

In [379]:
df_3

,VisitId,Products,Date,Rating
0,1606899638,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",2021-01-01,3.0
1,1606899665,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]",2021-01-01,10.4
2,1606922803,"[baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]",2021-01-01,57.0
3,1606959100,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",2021-01-01,4.0
4,1607000156,"[baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years)]",2021-01-01,0.0
...,...,...,...,...
8053,1617307082,"[food, non food, baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, pull-ups & swimming pants, baby pants]",2021-01-01,1.5
8054,1617307117,[food],2021-01-01,1.0
8055,1617307129,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years), changing & bathing, pull-ups & swimming pants, baby pants]",2021-01-01,1.0
8056,1617307178,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, milk, food & juices, baby & toddler formula, baby formula (6-12 months), baby formula (0-6 months)]",2021-01-01,2.0


In [380]:
df_3['ProductId'] = df_3['Products'].apply(lambda x: hash(tuple(x)))

In [428]:
df_3

,VisitId,Products,Date,Rating,ProductId
0,1606899638,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",2021-01-01,3.0,-2557524753552762064
1,1606899665,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]",2021-01-01,10.4,9184284246071064056
2,1606922803,"[baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]",2021-01-01,57.0,-3214438610066360808
3,1606959100,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",2021-01-01,4.0,-2557524753552762064
4,1607000156,"[baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years)]",2021-01-01,0.0,-388721366715840061
...,...,...,...,...,...
8053,1617307082,"[food, non food, baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, pull-ups & swimming pants, baby pants]",2021-01-01,1.5,-5703399678831698183
8054,1617307117,[food],2021-01-01,1.0,5163192443017674416
8055,1617307129,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years), changing & bathing, pull-ups & swimming pants, baby pants]",2021-01-01,1.0,-994684859816260667
8056,1617307178,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, milk, food & juices, baby & toddler formula, baby formula (6-12 months), baby formula (0-6 months)]",2021-01-01,2.0,-2979273346497778360


In [387]:
df_3['Date'] = pd.to_datetime(df_3['Date'])

In [450]:
df_3 = df_3.drop(columns=['Date', 'ProductId'])


In [466]:
df_3

,VisitId,Products,Rating
0,1606899638,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",3.0
1,1606899665,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]",10.4
2,1606922803,"[baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]",57.0
3,1606959100,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",4.0
4,1607000156,"[baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years)]",0.0
...,...,...,...
8053,1617307082,"[food, non food, baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, pull-ups & swimming pants, baby pants]",1.5
8054,1617307117,[food],1.0
8055,1617307129,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years), changing & bathing, pull-ups & swimming pants, baby pants]",1.0
8056,1617307178,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, milk, food & juices, baby & toddler formula, baby formula (6-12 months), baby formula (0-6 months)]",2.0


In [462]:
df_3['Products']

0                                                                                                               [baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]
1                                                                                                                     [baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]
2                                                                                                              [baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]
3                                                                                                               [baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]
4                                                                                                              [baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years)]
     

In [521]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from itertools import chain

# Flatten the product lists into individual rows
df_flat = df_3.explode('Products')

# Convert 0.0 values in Rating to 1 (purchased)
df_flat['Rating'] = df_flat['Rating'].replace(0.0, 1)

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the product descriptions into TF-IDF features
tfidf_matrix = tfidf_vectorizer.fit_transform(df_flat['Products'].astype(str))

# Compute the cosine similarity between products
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


In [526]:
# Function to get content-based recommendations for a product
def get_content_based_recommendations(product_name, df, cosine_similarities):
    product_index = df[df['Products'] == product_name].index[0]
    similar_products = list(enumerate(cosine_similarities[product_index]))

    # Sort the products based on similarity scores
    similar_products = sorted(similar_products, key=lambda x: x[1], reverse=True)

    # Exclude the input product from recommendations
    similar_products = [index for index, score in similar_products if index != product_index]

    # Filter out duplicate products (case-insensitive)
    unique_recommendations = set()
    filtered_recommendations = []
    for index in similar_products:
        product = df['Products'].iloc[index].lower()  # Convert to lowercase
        if product not in unique_recommendations:
            filtered_recommendations.append(df['Products'].iloc[index])
            unique_recommendations.add(product)

        # Break the loop if we have enough recommendations
        if len(filtered_recommendations) >= 10:
            break

    return filtered_recommendations[:10]

# Example: Get content-based recommendations for a product
product_name_to_recommend = 'natural baby diapers'
content_based_recommendations = get_content_based_recommendations(product_name_to_recommend, df_flat, cosine_similarities)

print(f"Top 5 content-based recommendations for {product_name_to_recommend}:")
pprint(content_based_recommendations)


Top 5 content-based recommendations for natural baby diapers:
['food & juices',
 'food &amp; juices',
 'food',
 'juices & beverages',
 'non food',
 'food cupboard',
 'fresh food',
 'baby products',
 'milk',
 'baby & toddler formula']


['Baby and Personal Care',
 'food cupboard',
 'baby wipes family pack',
 'baby &amp; toddler formula',
 'baby formula (1-6 years)',
 'non food',
 'Diapers',
 'jars & packets',
 'changing &amp; bathing',
 'سراويل الأطفال',
 'baby formula (6-12 months)',
 'baby formula (0-6 months)',
 'wipes & diaper cream',
 'Bulk Savings',
 'labneh',
 'food',
 'yoghurt',
 'Baby Products',
 'food &amp; juices',
 'natural baby wipes']
 
 ['pull-ups & swimming pants',
 'Baby and Personal Care',
 'food cupboard',
 'baby wipes family pack',
 'baby &amp; toddler formula',
 'baby formula (1-6 years)',
 'non food',
 'Diapers',
 'jars & packets',
 'changing &amp; bathing',
 'سراويل الأطفال',
 'baby formula (6-12 months)',
 'wipes & diaper cream',
 'Bulk Savings',
 'labneh',
 'food',
 'yoghurt',
 'Baby Products',
 'food &amp; juices',
 'natural baby wipes',
 'Pull-ups &amp; Swimming Pants',
 'changing & bathing',
 'baby & toddler formula',
 'Changing &amp; Bathing',
 'baby products',
 'حفاضات اطفال',
 'juices & beverages',
 'baby wipes single piece',
 'Baby Diapers',
 'diapers']

In [527]:
import pickle

# Save the TF-IDF vectorizer and cosine similarities to a pickle file
with open('model_1.pkl', 'wb') as file:
    pickle.dump((tfidf_vectorizer, cosine_similarities), file)


In [528]:
# Load the TF-IDF vectorizer and cosine similarities from the pickle file
with open('model_1.pkl', 'rb') as file:
    loaded_tfidf_vectorizer, loaded_cosine_similarities = pickle.load(file)


In [532]:
df_3.to_csv('recommended_data.csv', index=False)

In [531]:
df_3

,VisitId,Products,Rating
0,1606899638,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",3.0
1,1606899665,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]",10.4
2,1606922803,"[baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]",57.0
3,1606959100,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",4.0
4,1607000156,"[baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years)]",0.0
...,...,...,...
8053,1617307082,"[food, non food, baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, pull-ups & swimming pants, baby pants]",1.5
8054,1617307117,[food],1.0
8055,1617307129,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years), changing & bathing, pull-ups & swimming pants, baby pants]",1.0
8056,1617307178,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, milk, food & juices, baby & toddler formula, baby formula (6-12 months), baby formula (0-6 months)]",2.0


In [533]:
d_f = pd.read_csv('recommended_data.csv')

In [534]:
d_f

,VisitId,Products,Rating
0,1606899638,"['baby products', 'milk', 'food & juices', 'baby & toddler formula', 'baby formula (1-6 years)']",3.0
1,1606899665,"['baby products', 'changing & bathing', 'diapers', 'wipes & diaper cream', 'baby diapers']",10.4
2,1606922803,"['baby products', 'milk', 'food & juices', 'baby & toddler formula', 'baby formula (0-6 months)']",57.0
3,1606959100,"['baby products', 'milk', 'food & juices', 'baby & toddler formula', 'baby formula (1-6 years)']",4.0
4,1607000156,"['baby products', 'milk', 'food & juices', 'baby & toddler formula', 'baby formula (6-12 years)']",0.0
...,...,...,...
8053,1617307082,"['food', 'non food', 'baby products', 'changing & bathing', 'diapers', 'wipes & diaper cream', 'baby diapers', 'pull-ups & swimming pants', 'baby pants']",1.5
8054,1617307117,['food'],1.0
8055,1617307129,"['baby products', 'milk', 'food & juices', 'baby & toddler formula', 'baby formula (1-6 years)', 'changing & bathing', 'pull-ups & swimming pants', 'baby pants']",1.0
8056,1617307178,"['baby products', 'changing & bathing', 'diapers', 'wipes & diaper cream', 'baby diapers', 'milk', 'food & juices', 'baby & toddler formula', 'baby formula (6-12 months)', 'baby formula (0-6 months)']",2.0


In [535]:
df_3.to_pickle('data_recomend.pkl')

In [536]:
df_ = pd.read_pickle('data_recomend.pkl')

In [537]:
df_

,VisitId,Products,Rating
0,1606899638,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",3.0
1,1606899665,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]",10.4
2,1606922803,"[baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]",57.0
3,1606959100,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",4.0
4,1607000156,"[baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years)]",0.0
...,...,...,...
8053,1617307082,"[food, non food, baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, pull-ups & swimming pants, baby pants]",1.5
8054,1617307117,[food],1.0
8055,1617307129,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years), changing & bathing, pull-ups & swimming pants, baby pants]",1.0
8056,1617307178,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, milk, food & juices, baby & toddler formula, baby formula (6-12 months), baby formula (0-6 months)]",2.0


,VisitId,Products,Rating
0,1606899638,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",3.0
1,1606899665,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers]",10.4
2,1606922803,"[baby products, milk, food & juices, baby & toddler formula, baby formula (0-6 months)]",57.0
3,1606959100,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years)]",4.0
4,1607000156,"[baby products, milk, food & juices, baby & toddler formula, baby formula (6-12 years)]",0.0
...,...,...,...
8053,1617307082,"[food, non food, baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, pull-ups & swimming pants, baby pants]",1.5
8054,1617307117,[food],1.0
8055,1617307129,"[baby products, milk, food & juices, baby & toddler formula, baby formula (1-6 years), changing & bathing, pull-ups & swimming pants, baby pants]",1.0
8056,1617307178,"[baby products, changing & bathing, diapers, wipes & diaper cream, baby diapers, milk, food & juices, baby & toddler formula, baby formula (6-12 months), baby formula (0-6 months)]",2.0
